In [3]:
import os 
import requests
import pandas as pd

_data_root = './data/Diabetes'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'Diabetes.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1k5-1caezQ3zWJbKaiMULTGq-3sz6uThC'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

df_view= pd.read_csv("./data/Diabetes/Diabetes.csv")
df_view


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,100162476,AfricanAmerican,Male,[70-80),?,1,3,7,3,...,No,Down,No,No,No,No,No,Ch,Yes,>30
101762,443847782,74694222,AfricanAmerican,Female,[80-90),?,1,4,5,5,...,No,Steady,No,No,No,No,No,No,Yes,NO
101763,443854148,41088789,Caucasian,Male,[70-80),?,1,1,7,1,...,No,Down,No,No,No,No,No,Ch,Yes,NO
101764,443857166,31693671,Caucasian,Female,[80-90),?,2,3,7,10,...,No,Up,No,No,No,No,No,Ch,Yes,NO


In [1]:
import os
import mlflow
import requests
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer

df_view= pd.read_csv("./penguins_size.csv")


df_view.dropna(inplace=True)
df_view = df_view[df_view['sex']!='.']
df_view

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,MALE
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,FEMALE
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,FEMALE
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,FEMALE
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,MALE
...,...,...,...,...,...,...,...
338,Gentoo,Biscoe,47.2,13.7,214.0,4925.0,FEMALE
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,FEMALE
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,MALE
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,FEMALE


In [2]:
X = df_view.drop(columns=["species"])
y = df_view["species"]
text_cols = ["island", "sex"]
num_cols = [
    "culmen_length_mm", "culmen_depth_mm", "flipper_length_mm", "body_mass_g"
]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [3]:
preprocessor = ColumnTransformer(
    transformers=[
    ("text", OneHotEncoder(handle_unknown="ignore"), text_cols),
    ("num", StandardScaler(), num_cols)
])

modelRF = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [4]:
# Definición de la grilla de parámetros para el RandomForestClassifier
param_grid = {
    "classifier__n_estimators": [10, 50],
    "classifier__max_depth": [ 5],
    "classifier__min_samples_split": [2]
}

grid_search = GridSearchCV(
    estimator=modelRF,
    param_grid=param_grid,
    cv=5,               # Número de folds para validación cruzada
    scoring="accuracy", # Métrica de evaluación
    n_jobs=-1,          # Utiliza todos los cores disponibles
    verbose=2           # Muestra información del progreso
)

In [5]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://mlflow-minio:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("mlflow_penguins_vf5")

mlflow.sklearn.autolog(log_model_signatures=True, log_input_examples=True, registered_model_name="modelo_penguins_v2", max_tuning_runs=27)

with mlflow.start_run(run_name="autolog_pipe_model_reg") as run:
    grid_search.fit(X_train, y_train)

2025/04/26 16:28:30 INFO mlflow.tracking.fluent: Experiment with name 'mlflow_penguins_vf5' does not exist. Creating a new experiment.
2025/04/26 16:28:31 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    ex

Fitting 5 folds for each of 2 candidates, totalling 10 fits


Successfully registered model 'modelo_penguins_v2'.
2025/04/26 16:28:39 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: modelo_penguins_v2, version 1
Created version '1' of model 'modelo_penguins_v2'.
2025/04/26 16:28:41 INFO mlflow.sklearn.utils: Logging the 27 best runs, no runs will be omitted.


[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=10; total time=   0.0s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=10; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=10; total time=   0.0s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=50; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n_estimators=10; total time=   0.1s
[CV] END classifier__max_depth=5, classifier__min_samples_split=2, classifier__n

In [1]:
import mysql.connector
from mysql.connector import Error

# Configura los datos de conexión a tu servicio MySQL
config = {
    'host': 'mysql-data-store',   
    'user': 'admin',        
    'password': 'supersecret', 
    'database': 'datadb'
}

try:
    connection = mysql.connector.connect(**config)

    if connection.is_connected():
        cursor = connection.cursor()
        
        create_table_query = """
        CREATE TABLE IF NOT EXISTS resultados (
            id INT AUTO_INCREMENT PRIMARY KEY,
            island VARCHAR(255),
            culmen_length_mm FLOAT,
            culmen_depth_mm FLOAT,
            flipper_length_mm FLOAT,
            body_mass_g FLOAT,
            sex VARCHAR(50),
            model VARCHAR(50),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """

        cursor.execute(create_table_query)
        connection.commit()
        print("Tabla 'penguins' creada exitosamente.")

except Error as e:
    print(f"Error al conectar o crear la tabla: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()


Tabla 'penguins' creada exitosamente.


In [2]:
import psycopg2

try:
    # Conexión a la base de datos PostgreSQL
    connection = psycopg2.connect(
        host='postgres-data-store',   # El nombre de tu servicio en docker-compose
        database='datadb',            # La base de datos que configuraste
        user='admin',                 # El usuario configurado
        password='supersecret'        # La contraseña
    )

    cursor = connection.cursor()

    # Script para crear la tabla
    create_table_query = """
    CREATE TABLE IF NOT EXISTS resultados_modelo (
        id SERIAL PRIMARY KEY,
        island VARCHAR(255),
        culmen_length_mm FLOAT,
        culmen_depth_mm FLOAT,
        flipper_length_mm FLOAT,
        body_mass_g FLOAT,
        sex VARCHAR(50),
        model VARCHAR(50),
        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
    );
    """

    cursor.execute(create_table_query)
    connection.commit()
    print("Tabla 'Resultados' creada exitosamente en PostgreSQL.")

except Exception as e:
    print(f"Error al conectar o crear la tabla: {e}")

finally:
    if connection:
        cursor.close()
        connection.close()


Tabla 'Resultados' creada exitosamente en PostgreSQL.
